In [16]:
import subprocess
import math
from pydub import AudioSegment



def extract_audio_from_video(video_path, audio_path):

    command = [
        "ffmpeg",
        "-i",
        video_path,
        "-vn",
        audio_path,
    ]

    subprocess.run(command)


extract_audio_from_video(
    "./files/DP_Longest_Common_Subsequence_Leetcode1143.mp4", "./files/"
)


def cut_audio_in_chunks(audio_path, chunk_size, chunks_folder):
    track = AudioSegment.from_mp3(audio_path)
    chunk_len = chunk_size * 60 * 1000
    chunks = math.ceil(len(track) / chunk_len)
    for i in range(chunks):
        start_time = i * chunk_len
        end_time = (i + 1) * chunk_len
        chunk = track[start_time:end_time]

        chunk.export(f"{chunks_folder}/chunk_{i}.mp3", format="mp3")

In [17]:
cut_audio_in_chunks(
    "./files/DP_Longest_Common_Subsequence_Leetcode1143.mp3", 10, "./files/chunks"
)

In [22]:
import openai
import glob


def transcribe_chunks(chunk_folder, destination):
    files = glob.glob(f"{chunk_folder}/*.mp3")
    final_transcript = ""
    # find mp3 files in the folder
    for file in files:
        # get transcript
        with open(file, "rb") as audio_file:
            transcript = openai.Audio.transcribe(
                "whisper-1",
                audio_file,
            )
            # add it to the final transcript
            final_transcript += transcript["text"]

    # And open a file, and write the final transcript
    with open(destination, "w") as file:
        file.write(final_transcript)


transcribe_chunks("./files/chunks", "./files/chunks/transcript.txt")